In [1]:
pip install tensorflow numpy matplotlib keras

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

def load_and_preprocess_data():
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
    train_images = train_images.astype("float32") / 255
    test_images = test_images.astype("float32") / 255
    train_images = np.expand_dims(train_images, -1)
    test_images = np.expand_dims(test_images, -1)
    num_classes = 10
    train_labels = keras.utils.to_categorical(train_labels, num_classes)
    test_labels = keras.utils.to_categorical(test_labels, num_classes)

    return train_images, train_labels, test_images, test_labels
def build_model():
    model = keras.Sequential(
        [
            keras.Input(shape=(28, 28, 1)),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(10, activation="softmax"),
        ]
    )

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

def train_model(model, train_images, train_labels, test_images, test_labels):
    batch_size = 128
    epochs = 15

    history = model.fit(
        train_images,
        train_labels,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.1
    )

    score = model.evaluate(test_images, test_labels, verbose=0)
    print(f"Test loss: {score[0]}")
    print(f"Test accuracy: {score[1]}")

    return history


def visualize_predictions(model, test_images, test_labels):
    test_labels = np.argmax(test_labels, axis=1)
    predictions = model.predict(test_images)
    predicted_labels = np.argmax(predictions, axis=1)
    plt.figure(figsize=(10, 10))
    for i in range(25):
        plt.subplot(5, 5, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(test_images[i].squeeze(), cmap=plt.cm.binary)
        plt.xlabel(f"Pred: {predicted_labels[i]}, True: {test_labels[i]}")
    plt.show()

def main():

    train_images, train_labels, test_images, test_labels = load_and_preprocess_data()
    model = build_model()
    model.summary()
    history = train_model(model, train_images, train_labels, test_images, test_labels)
    visualize_predictions(model, test_images, test_labels)
    model.save("mnist_model.h5")

if __name__ == "__main__":
    main()